# Data Preparation

## Extract Data
Data Source URL: https://beta.data.gov.sg/collections/189/view

API Documentation: https://guide.data.gov.sg/developer-guide/api-v2

In [53]:
import pandas as pd
import requests

In [85]:
def extract_data() -> pd.DataFrame:
    offset_limit = 0
    df = []
    url = 'https://data.gov.sg/api/action/datastore_search?resource_id=d_8b84c4ee58e3cfc0ece0d773c8ca6abc'
    
    # Loop through the results and add if present (the default max number of rows to return is 100)
    while True:
        querystring = {"offset": "{}".format(offset_limit), "filters": "{\"flat_type\": \"4 ROOM\"}", "limit": "5000"}
        response = requests.get(url=url, params=querystring)
        data = response.json()['result']['records']

        # If there are no more records
        if len(data) == 0:
            break
        # else add the data to the main list df
        df.extend(data)
        # Increase offset_limit to skip the already added data
        offset_limit = offset_limit + 5000

    return pd.DataFrame(df).drop(columns=['_id'])

## Transform Data

In [1]:
import numpy as np

In [85]:
def transform_data(df: pd.DataFrame) -> pd.DataFrame:
    
    # Change the unit of `remaining_lease` from years-months to months
    years = df['remaining_lease'].apply(lambda x: pd.to_numeric(x[0:2]))
    months = df['remaining_lease'].apply(lambda x: pd.to_numeric(x[9:11]) if len(x) > 8 else pd.to_numeric('00'))
    df['remaining_lease'] = (years * 12) + months
    
    # Merge 26 towns into 5 bigger regions
    central = ['CENTRAL AREA', 'BUKIT TIMAH']
    rest_of_central = ['BISHAN', 'GEYLANG', 'KALLANG/WHAMPOA','MARINE PARADE','QUEENSTOWN','TOA PAYOH','BUKIT MERAH']
    near_central = ['CLEMENTI','BUKIT BATOK','BUKIT PANJANG','ANG MO KIO','SERANGOON','HOUGANG','BEDOK']
    further_central = ['JURONG EAST','CHOA CHU KANG','SENGKANG','YISHUN','TAMPINES']
    far_central = ['JURONG WEST','PASIR RIS','PUNGGOL','SEMBAWANG','WOODLANDS']
    region = [central, rest_of_central, near_central, further_central, far_central]
    def region_finder(x):
        for i in range(4):
            if x in region[i]:
                break
        return i
    df['proximity_to_ctr'] = df['town'].apply(region_finder)
    
    # Create new variable `central` (1 if the flat is located in the Central region, 0 otherwise)
    df['central'] = df['proximity_to_ctr'].apply(lambda x: 1 if (x < 2) else 0)
    
    # Create new variable `average_level` from `storey_range` (format "xx TO xx" e.g. "10 TO 12" means level 10/11/12)
    def range_to_lvl(x):
        x = (int(x[:2]) + int(x[7:])) / 2
        return x
    df['average_level'] = df['storey_range'].apply(range_to_lvl)
    
    # Create new variable `n_hospitals` representing the number of hospitals in the same town as the flat observed
    cond = [(df['town'] == 'ANG MO KIO'), (df['town'] == 'BEDOK'), (df['town'] == 'BISHAN'), (df['town'] == 'BUKIT BATOK'),
            (df['town'] == 'BUKIT MERAH'), (df['town'] == 'BUKIT PANJANG'), (df['town'] == 'BUKIT TIMAH'),
            (df['town'] == 'CENTRAL AREA'), (df['town'] == 'CHOA CHU KANG'), (df['town'] == 'CLEMENTI'),
            (df['town'] == 'GEYLANG'), (df['town'] == 'HOUGANG'), (df['town'] == 'JURONG EAST'), (df['town'] == 'JURONG WEST'),
            (df['town'] == 'KALLANG/WHAMPOA'), (df['town'] == 'MARINE PARADE'), (df['town'] == 'PASIR RIS'), 
            (df['town'] == 'PUNGGOL'), (df['town'] == 'QUEENSTOWN'), (df['town'] == 'SEMBAWANG'), (df['town'] == 'SENGKANG'),
            (df['town'] == 'SERANGOON'), (df['town'] == 'TAMPINES'), (df['town'] == 'TOA PAYOH'), (df['town'] == 'WOODLANDS'),
            (df['town'] == 'YISHUN')]
    val = [2, 0, 1, 1, 3, 0, 7, 3, 0, 1, 0, 1, 2, 0, 0, 1, 0, 3, 1, 0, 3, 1, 2, 0, 0, 2]
    df['n_hospitals'] = np.select(cond, val)
    
    # Create new variable `n_mrt` representing the number of MRT stations in the same town as the flat observed
    cond2 = [(df['town'] == 'ANG MO KIO'), (df['town'] == 'BEDOK'), (df['town'] == 'BISHAN'), (df['town'] == 'BUKIT BATOK'),
            (df['town'] == 'BUKIT MERAH'), (df['town'] == 'BUKIT PANJANG'), (df['town'] == 'BUKIT TIMAH'),
            (df['town'] == 'CENTRAL AREA'), (df['town'] == 'CHOA CHU KANG'), (df['town'] == 'CLEMENTI'),
            (df['town'] == 'GEYLANG'), (df['town'] == 'HOUGANG'), (df['town'] == 'JURONG EAST'), (df['town'] == 'JURONG WEST'),
            (df['town'] == 'KALLANG/WHAMPOA'), (df['town'] == 'MARINE PARADE'), (df['town'] == 'PASIR RIS'), 
            (df['town'] == 'PUNGGOL'), (df['town'] == 'QUEENSTOWN'), (df['town'] == 'SEMBAWANG'), (df['town'] == 'SENGKANG'),
            (df['town'] == 'SERANGOON'), (df['town'] == 'TAMPINES'), (df['town'] == 'TOA PAYOH'), (df['town'] == 'WOODLANDS'),
            (df['town'] == 'YISHUN')]
    val2 = [3, 5, 4, 2, 7, 2, 8, 23, 2, 1, 8, 3, 4, 1, 4, 3, 1, 1, 9, 2, 0, 2, 6, 4, 4, 3]
    df['n_mrt'] = np.select(cond2, val2)

    return pd.DataFrame(df).drop(columns=['town', 'storey_range', 'month', 'street_name'])

## Load Data

In [16]:
def load_data(df: pd.DataFrame):
    df.to_csv(r'resale-flat-prices-updated.csv')

In [ ]:
data = extract_data()
data = transform_data(data)
load_data(data)